In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [33]:
hashtag='instagram'
browser = webdriver.Chrome()
browser.get('https://www.instagram.com/'+hashtag)
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [34]:
#Extract links from user profile page
links=[]
source = browser.page_source
data=bs(source, 'html.parser')
body = data.find('body')
script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
# print(script)
page_json = script.string.split(' = ', 1)[1].rstrip(';')
data = json.loads(page_json)


In [36]:
#try 'script.string' instead of script.text if you get error on index out of range
for link in data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']:
    links.append('https://www.instagram.com'+'/p/'+link['node']['shortcode']+'/')
#try with ['display_url'] instead of ['shortcode'] if you don't get links 
#Extract links from hashtag page
print(links)


['https://www.instagram.com/p/CFPtx6-MMr5/', 'https://www.instagram.com/p/CFPmeKrjVDW/', 'https://www.instagram.com/p/CFND8w3s17W/', 'https://www.instagram.com/p/CFKs92bsm_d/', 'https://www.instagram.com/p/CFKrTnzjm2S/', 'https://www.instagram.com/p/CFKffKOsxCF/', 'https://www.instagram.com/p/CFH3WKBMZj8/', 'https://www.instagram.com/p/CFFTClrMW6h/', 'https://www.instagram.com/p/CFAq350jcXz/', 'https://www.instagram.com/p/CE94U12MNVs/', 'https://www.instagram.com/p/CE9xgF3jwS_/', 'https://www.instagram.com/p/CE7AhQ9jlQv/']


In [50]:
hashtag='beardcare'
browser = webdriver.Chrome()
browser.get('https://www.instagram.com/explore/tags/'+hashtag)
# Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
scroll(browser, 5)

links=[]
source = browser.page_source
data=bs(source, 'html.parser')
body = data.find('body')
script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
page_json = script.string.split(' = ', 1)[1].rstrip(';')
data = json.loads(page_json)
for link in data['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
    links.append('https://www.instagram.com'+'/p/'+link['node']['shortcode']+'/')

KeyboardInterrupt: 

In [52]:
len(links)

69

In [49]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [46]:
links

['https://www.instagram.com/p/CFRJBoeAf09/',
 'https://www.instagram.com/p/CFRI6v1AcKG/',
 'https://www.instagram.com/p/CFRIoOuo9t5/',
 'https://www.instagram.com/p/CFRIXfCFVG0/',
 'https://www.instagram.com/p/CFRHqw-nAa8/',
 'https://www.instagram.com/p/CFRHZzCHUbg/',
 'https://www.instagram.com/p/CFRHJ7CBgBC/',
 'https://www.instagram.com/p/CFRG6obBUIn/',
 'https://www.instagram.com/p/CFRGGKhg6Ug/',
 'https://www.instagram.com/p/CFRF17UKWLi/',
 'https://www.instagram.com/p/CFRFtSYDj6T/',
 'https://www.instagram.com/p/CFRD4N7AF3W/',
 'https://www.instagram.com/p/CFRD1qIBBdF/',
 'https://www.instagram.com/p/CFRCIiSBkYO/',
 'https://www.instagram.com/p/CFRB2jFBTOt/',
 'https://www.instagram.com/p/CFRBw_QF9qX/',
 'https://www.instagram.com/p/CFRBgvunRva/',
 'https://www.instagram.com/p/CFQ_sNTnR20/',
 'https://www.instagram.com/p/CFQ_JcsBT6K/',
 'https://www.instagram.com/p/CFQ-IRTjSPt/',
 'https://www.instagram.com/p/CFQ-BPSHLGx/',
 'https://www.instagram.com/p/CFQ9fZEgZIt/',
 'https://

In [54]:
from instaloader import Instaloader, Profile
import datetime


C:\Users\arane\anaconda3\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)


In [70]:
MAX_DAYS = 50

LIKES_WEIGHT = 1
COMMENTS_WEIGHT = 1
NUM_FOLLOWERS_WEIGHT = 1
NUM_POSTS_WEIGHT = 1

def get_summary(profile):
    user = {}
    # print('Engagement.get_summary: {}'.format(profile.username))
    user['followers'] = profile.followers
    # print('  Followers: {}'.format(profile.followers))

    total_num_likes = 0
    total_num_comments = 0
    total_num_posts = 0
    current_date = datetime.datetime.now()
    location = []

    for post in profile.get_posts():
        delta = current_date - post.date
        if (delta.days > MAX_DAYS):
            break
        if (post.likes is not None):
            total_num_likes += post.likes
        if (post.comments is not None):
            total_num_comments += post.comments
        location.append(profile.get_locations())
        total_num_posts += 1
    
        # print('  {} - Number of Likes: {}, Number of Comments: {}, Post Date: {}'.format(total_num_posts, total_num_likes, total_num_comments, post.date))

    engagement = 0
    if profile.followers > 0 and total_num_posts > 0:
        engagement = float( (LIKES_WEIGHT * total_num_likes) + (COMMENTS_WEIGHT * total_num_comments)) / ((NUM_FOLLOWERS_WEIGHT * profile.followers) * (NUM_POSTS_WEIGHT * total_num_posts))
    user['engagement'] = engagement * 100
    # print('  Engagement: {}'.format(user['engagement']))
    user['num_recent_posts'] = total_num_posts
    
    user['location'] = location
    # print('  Number of Recent Posts: {}'.format(user['num_recent_posts']))
    post_freq = 0.0
    if (total_num_posts > 0):
        post_freq = float(MAX_DAYS) / total_num_posts
    user['post_frequency'] = post_freq
    # print('  Recent Post Frequency: {}'.format(user['post_frequency']))
    return user

In [71]:
loader = Instaloader()
NUM_POSTS = 10

def get_hashtags_posts(query):
    posts = loader.get_hashtag_posts(query)
    users = {}
    count = 0
    for post in posts:
        profile = post.owner_profile
        if profile.username not in users:
            summary = engagement.get_summary(profile)
            users[profile.username] = summary
            count += 1
            print('{}: {}'.format(count, profile.username))
            if count == NUM_POSTS:
                break
    return users

In [72]:
import threading
from instaloader import Instaloader, Profile
# import engagement
import pickle

loader = Instaloader()
NUM_POSTS = 10

def get_hashtags_posts(query):
    posts = loader.get_hashtag_posts(query)
    users = {}
    count = 0
    for post in posts:
        profile = post.owner_profile
        if profile.username not in users:
            summary = get_summary(profile)
            users[profile.username] = summary
            count += 1
            print('{}: {}'.format(count, profile.username))
            if count == NUM_POSTS:
                break
    return users

if __name__ == "__main__":
    hashtag = "beardcare"
    users = get_hashtags_posts(hashtag)
    print(users)

AttributeError: 'Profile' object has no attribute 'get_locations'